# Морфология
В этом ноутбуке описана подготовка данных для задачи POS-tagging. А также пара простых моделей на keras, решающих данную задачу. Оригинальная задача и ноутбук есть в контесте: https://www.kaggle.com/c/rupos2018/overview

## Часть 1. Загрузка корпуса
Здесь мы прочитаем корпуса из csv и разложим их по спискам.

In [2]:
# для совместимости со вторым питоном
from __future__ import print_function
import io

In [3]:
# Имена файлов с данными.
TRAIN_FILENAME = "data/input/train.csv"
TEST_FILENAME = "data/input/test.csv"

In [4]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple("WordForm", "word pos gram")

def get_sentences(filename, is_train):
    sentences = []
    with io.open(filename, "r", encoding='utf-8') as r:
        # Пропускаем заголовок
        next(r)
        sentence = [] # будем заполнять список предложений
        for line in r:
            # предложения отделены по '\n'
            if len(line.strip()) == 0:
                if len(sentence) == 0:
                    continue
                sentences.append(sentence)
                sentence = []
                continue
            if is_train:
                # Формат: индекс\tномер_в_предложении\tсловоформа\tPOS#Грамемы
                word = line.strip().split("\t")[2]
                pos = line.strip().split("\t")[3].split("#")[0]
                gram = line.strip().split("\t")[3].split("#")[1]
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.strip().split("\t")[2]
                sentence.append(word)
        if len(sentence) != 0:
            sentences.append(sentence)
    return sentences

In [5]:
train = get_sentences(TRAIN_FILENAME, True)
test = get_sentences(TEST_FILENAME, False)

In [6]:
# Выыедем, что получилось
for wordform in train[0][:10]:
    print(wordform.word, '\t', wordform.pos, '\t', wordform.gram)

А 	 CONJ 	 _
ведь 	 PART 	 _
для 	 ADP 	 _
конкретных 	 ADJ 	 Case=Gen|Degree=Pos|Number=Plur
изделий 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur
зачастую 	 ADV 	 Degree=Pos
нужен 	 ADJ 	 Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev
монокристалл 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
не 	 PART 	 _
только 	 PART 	 _


Для простоты далее будем использовать токены слов и POS-теги. Но чтобы определять грамматические значения нужно еще провести некоторые манипуляции с данными, описанные в оригинальном ноутубке. Мы же ограничимся только определением частей речи

## Часть 2. Подготовка эмбеддингов

Обычно в качестве признаков для обучения сеток используются словные эмбеддинги. Для этого можно скачать предобученные и сохранить их в матрицу, где в расположатся векторы эмбеддингах по индексам, соответсвующих слов

In [7]:
#запомним все уникальные слова и POS-теги в корпусе
word_set = set()
pos_set = set()
for sent in train:
    for wordform in sent:
        word_set.add(wordform.word.lower())
        pos_set.add(wordform.pos)

In [8]:
for word in list(word_set)[:10]: 
    print(word, end=', ')
print()
print(pos_set)

подлежала, уральское, бюргерам, коваными, девятого, уральском, правильную, медном, страданием, изношенного, 
set([u'ADV', u'NOUN', u'ADP', u'PRON', u'SCONJ', u'PROPN', u'DET', u'SYM', u'INTJ', u'PUNCT', u'PART', u'X', u'AUX', u'NUM', u'CONJ', u'ADJ', u'VERB'])


In [9]:
#Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним
import numpy as np

word_embeddings_path = 'data\glove.6B.50d.txt'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with io.open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PADDING_TOKEN"] = len(word2idx)
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNKNOWN_TOKEN"] = len(word2idx)
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        #Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue
        
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

In [10]:
len(word_set & set(word2idx.keys()))

1947

In [11]:
len(word_set)

98880

Как-то эмбеддинги не сильно подходят для данного корпуса поэтому, просто инициализируем рандмно матрицу эмбеддингов при определении сетки. Вам же предлагается все-таки поискать подходящие эмбеддинги и использовать их при обучении.

## Часть 3. Подготовка данных
Теперь нам остается только пронумеровать все слова и POS-теги и можно переходить к обучению сеток.

In [12]:
word_to_index = {'PAD' : 0, 'UNK' : 1}
for word in word_set:
    word_to_index[word] = len(word_to_index)

In [13]:
pos_to_index = {}
index_to_pos = {}
for pos in pos_set:
    pos_to_index[pos] = len(pos_to_index)
    index_to_pos[len(index_to_pos)] = pos

In [14]:
# для полносвязной сетки просто захреначим все в один список
data_X = []
data_Y = []
for sent in train:
    for wordform in sent:
        data_X.append(word_to_index[wordform.word.lower()])
        data_Y.append(pos_to_index[wordform.pos])

In [16]:
print(data_X[:10])
print(data_Y[:10])

[43904, 87202, 73490, 80225, 49180, 73764, 31229, 5478, 65911, 66276]
[14, 10, 2, 15, 1, 0, 15, 1, 10, 10]


## Часть 4. Полносвязная сеть
Самой простой моделью является обычный перцептрон. На вход сетки будем подавать просто эмдеддинг каждого слова, на выходе ожидать распредедение вероятностей по тегам. В качестве фреймворка достаточно будет использовать keras и его Sequential модель (https://keras.io/models/sequential/), в которую слои добавляются последовательно, с помощью метода `add`.

In [30]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Activation, Flatten

In [47]:
model = Sequential()
# на самом деле на вход сетки будет добавляться индекс слова, а слой эмбеддинга будет возвращать для него вектор
model.add(Embedding(input_length=1, input_dim=len(word_to_index), output_dim=50, embeddings_initializer='random_uniform',
                    trainable=False)) # матрицу эмбеддингов просто инициализируем нормальным распределением и отключим обучение
# далее нам нужно схлопнуть трехмерный тензор с одной фиктивной размерностью в двумерный
model.add(Flatten())
model.add(Dense(100)) # основной полносвязный слой
model.add(Activation('relu')) # для приличия добавим функцию активации
model.add(Dense(len(pos_to_index))) # выходной слой тоже полносвязный размерности по кол-ву тегов
model.add(Activation('softmax')) # ну и в конце делаем softmax, чтобы получить распределение
model.summary() # вывод получившейся модели

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1, 50)             4944100   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               5100      
_________________________________________________________________
activation_8 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 17)                1717      
_________________________________________________________________
activation_9 (Activation)    (None, 17)                0         
Total params: 4,950,917
Trainable params: 6,817
Non-trainable params: 4,944,100
________________________________________

In [48]:
# компилируем модель
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [50]:
# и обучаем
model.fit(np.array(data_X), np.array(data_Y), epochs=1, batch_size=256)

Epoch 1/1
850689/850689 [==============================] - 4953s 6ms/step - loss: 1.4284 - accuracy: 0.5380


Проверка обученности модели остается за вами. Этот пример лишь для того, чтобы показать как собрать сетку и скормить ей данные.

## Часть 5. Рекуррентая сеть.

Далее рассмотрим более приближенную к SOTA модель. Ей является рекуррентая сеть, которая принимает эмбеддинги слов в предложении и генерирует для них распределение вероятностей. Основным отличием от прошлой в том, что теперь мы будем использовать соседние слова как раз за счет рекуррентого слоя. Для этой модели мы уже будем использовать функциональный способ задания модели все того же кераса (https://keras.io/models/model/).

In [51]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input
from keras.models import Model

In [52]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(None,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=len(word_to_index), output_dim=50, 
                             trainable=False, embeddings_initializer='random_uniform',
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(blstm_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          4944100   
_________________________________________________________________
blstm (Bidirectional)        (None, None, 200)         120800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 17)          3417      
Total params: 5,068,317
Trainable params: 124,217
Non-trainable params: 4,944,100
_________________________________________________________________


Далее нам нужно было бы распределить слова по предложениям, распределить по группам по длине, выравнить предложения по длине в одной групе, заполнив недостающие слова паддингами. Но это довольно неприятный процесс, а мне просто хочется запустить сетку и проверить, что она вообще работает, что сошлись все разверности. Поэтому просто раскидаем по 10 слов с помощью `numpy.reshape`

In [63]:
rnnX = np.reshape(data_X[:850000], (-1,10))
rnnY = np.reshape(data_Y[:850000], (-1,10,1))

In [58]:
np.shape(rnnX)

(85000L, 10L, 1L)

Ну и проверим, что оно вообще работает.

In [65]:
model.fit(rnnX, rnnY, epochs=1, batch_size=256)

Epoch 1/1
85000/85000 [==============================] - 26455s 311ms/step - loss: 2.0810 - accuracy: 0.3562


## Часть 6. Задание
В качестве упражения предлагается довести до ума обучения второй модели: распределить слова по предложениям, написать тестирование модели и собственно посмотреть как оно обучилось. Тестировать предлагаю на последней 1000 предложений, обучать - на остальном. Кто уверен в своих желаниях, то может решить оригинальную задачу: предсказывать также грамматические категории. 